In [1]:
import sys

sys.path.append("../")

%load_ext autoreload
%autoreload 2

import random

import numpy as np
import pandas as pd
from simulator.model.robust_mse_2 import RobustBidMSE_CTRCVR
from simulator.model.simple import SimpleBid
from simulator.simulation.utils_visualization import plot_2d_heatmaps_interpolated, plot_metric_with_error
from simulator.validation.check_results import autobidder_check
from tqdm import tqdm

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', None)

# This notebook provides a guideline how to simulate and check Robust PID Bidder for all the campaigns in the dataset
### This is a case of double (CTR+CVR) uncertainty

In [2]:
loss_type = 'MSE'

# Load data

In [3]:
auction_mode = 'FPA'

# data paths
campaigns_path = f"../data/data/{auction_mode.lower()}/campaigns_{auction_mode.lower()}_filtered_train_final.csv"
stats_path = f"../data/data/{auction_mode.lower()}/stats_{auction_mode.lower()}_filtered_train_final.csv"
STATS_PATH_NOISED = f"../data/data/{auction_mode.lower()}/stats_{auction_mode.lower()}_filtered_train_noised_CTR_CVR.csv"

In [4]:
campaign_df = pd.read_csv(campaigns_path)
stats_df = pd.read_csv(stats_path)

In [5]:
campaigns_path_big = f"../data/{auction_mode.lower()}/campaigns.csv"
campaign_big = pd.read_csv(campaigns_path)

In [6]:
campaign_df.describe()

,Unnamed: 0,campaign_id,loc_id,item_id,campaign_start,campaign_end,auction_budget,microcat_ext,logical_category,region_id,start_hour
count,317.000000,3.170000e+02,317.000000,3.170000e+02,3.170000e+02,3.170000e+02,317.000000,3.170000e+02,317.000000,317.000000,317.000000
mean,158.000000,7.221170e+07,643730.753943,3.464000e+09,5.295671e+08,5.296535e+08,482.268864,1.310064e+06,2.669811,642908.586751,6.599369
std,91.654242,4.545295e+06,13312.992329,7.972316e+08,8.416271e+03,8.416271e+03,653.280628,9.463114e+05,1.465637,9740.578484,3.776225
min,0.000000,1.379587e+07,621630.000000,2.944305e+08,5.295350e+08,5.296214e+08,3.840000,1.800000e+01,1.000000,621590.000000,0.000000
25%,79.000000,7.287904e+07,637640.000000,2.891171e+09,5.295645e+08,5.296509e+08,134.400000,4.473000e+04,1.150000,637640.000000,5.000000
50%,158.000000,7.299197e+07,639740.000000,3.820329e+09,5.295691e+08,5.296555e+08,268.800000,1.289833e+06,3.100000,637680.000000,6.000000
75%,237.000000,7.311409e+07,653140.000000,4.058614e+09,5.295726e+08,5.296590e+08,614.400000,2.169972e+06,3.410000,652560.000000,7.000000
max,316.000000,7.327750e+07,806540.000000,4.577361e+09,5.295773e+08,5.296637e+08,6669.120000,2.300143e+06,5.370000,662530.000000,23.000000


In [7]:
stats_df['CVR'] = stats_df['CRPredicts']

In [8]:
stats_df.describe()

,campaign_id,period,item_id,contact_price_bin,AuctionVisibilitySurplus,AuctionClicksSurplus,AuctionContactsSurplus,AuctionWinBidSurplus,CTRPredicts,CRPredicts,AuctionCount,x_platfrom_num,CTRPredicts_noised,CVR
count,1.998590e+05,1.998590e+05,1.998590e+05,199859.000000,1.998590e+05,1.998590e+05,1.998590e+05,199859.000000,199859.000000,199859.000000,199859.000000,37931.000000,199859.000000,199859.000000
mean,7.217870e+07,5.296049e+08,3.453895e+09,37.570087,2.292433e+00,1.985176e-01,1.606526e-02,253.203315,0.040505,0.098286,14.537314,19.625531,0.055559,0.098286
std,4.690015e+06,2.726080e+04,7.986468e+08,14.994818,7.123523e+00,5.280265e-01,4.728542e-02,942.599109,0.039297,0.097094,40.122623,29.471370,0.043702,0.097094
min,1.379587e+07,5.295348e+08,2.944305e+08,-44.000000,4.057939e-07,4.759838e-08,6.842122e-09,0.000000,0.001700,0.003221,1.000000,1.000000,0.010000,0.003221
25%,7.287402e+07,5.295816e+08,2.888165e+09,30.000000,1.446765e-01,1.195774e-02,5.472256e-04,0.920000,0.015004,0.027795,3.000000,6.000000,0.010000,0.027795
50%,7.299197e+07,5.296032e+08,3.815980e+09,41.000000,5.714305e-01,4.944516e-02,2.595186e-03,28.690000,0.024219,0.058972,5.000000,11.000000,0.058031,0.058972
75%,7.311387e+07,5.296248e+08,4.058614e+09,48.000000,1.861864e+00,1.784186e-01,1.129214e-02,175.150000,0.052420,0.144046,13.000000,21.000000,0.100000,0.144046
max,7.327750e+07,5.296608e+08,4.577361e+09,77.000000,3.751635e+02,3.718276e+01,1.909585e+00,72046.960000,0.633199,0.823119,1967.000000,270.000000,0.100000,0.823119


## Robust LP Bid vs LP Bid

In [9]:
def create_noised_stats_mse(stats_df, old_ctr, eps_ctr, eps_cvr, auction_mode, seed):
    random.seed(seed)
    np.random.seed(seed)

    grouped = stats_df.groupby('campaign_id')
    for campaign_id, group in grouped:
        old_ctr = group['CTRPredicts'].values
        noise = np.random.rand(old_ctr.size)
        noise = noise / np.linalg.norm(noise) * np.sqrt(2 * eps_ctr)
        stats_df.loc[stats_df.campaign_id == campaign_id, 'CTRPredicts_noised'] = np.clip(old_ctr + noise, 0.01, 0.1)

        old_cvr = group['CRPredicts'].values
        noise = np.random.rand(old_cvr.size)
        noise = noise / np.linalg.norm(noise) * np.sqrt(2 * eps_cvr)
        stats_df.loc[stats_df.campaign_id == campaign_id, 'CRPredicts_noised'] = np.clip(old_cvr + noise, 0.001, 0.4)

    stats_df.to_csv(STATS_PATH_NOISED)


In [10]:
robust_times = []
risk_bid_times = []
non_robust_times = []

In [11]:
eps_set_ctr = [10., 5., 1., 0.1, 0.01, 0.001, 1e-3, 1e-4, 1e-5] # [5., 1., 0.5, 0.1, 0.01, 0.001, 1e-3, 5e-4, 1e-4, 5e-5, 1e-5]
eps_set_cvr = [10., 5., 1., 0.1, 0.01, 0.001, 1e-3, 1e-4, 1e-5] # [0.5, 0.1, 0.01, 0.001, 1e-3, 1e-4, 1e-5]
bidder_types = ['simple', 'robust']
seeds = list(range(5))

# metrics_df = pd.DataFrame(columns=['eps', 'bidder_type', 'seed', 'tvc', 'cpc_percent', 'cpc_avg'])
metrics_list = []
T = 10
old_ctr = np.array(stats_df.CTRPredicts.copy())
T_times = {}
T_times_non_robust = {}
for eps_ctr in eps_set_ctr:
    for eps_cvr in eps_set_cvr:
        # for T in [1, 10, 100, 1000, int(1e4), int(1e5)]:
        robust_times = []
        non_robust_times = []
        for seed in tqdm(seeds):
            create_noised_stats_mse(stats_df, old_ctr, eps_ctr, eps_cvr, auction_mode, seed)

            CPC = 100.
            # start_non_robust = time.time()
            # Simple bid
            res_simple = autobidder_check(
                bidder=SimpleBid,
                params={"input_campaigns": campaigns_path,
                        "input_stats": STATS_PATH_NOISED,
                        'eps_ctr': eps_ctr,
                        'eps_cvr': eps_cvr,
                        'p': 1,
                        'q': 1,
                        'LP': True,
                        'CPC': CPC},
                loss_type=loss_type
            )
            # end_non_robust = time.time()
            # non_robust_times.append(start_non_robust-end_non_robust)
            times_campaigns_nonrobust = sum(res_simple['mean_times']) / len(res_simple['mean_times'])
            non_robust_times.append(times_campaigns_nonrobust)
            metrics_list.append({
                'eps_ctr': eps_ctr,
                'eps_cvr': eps_cvr,
                'bidder_type': 'simple',
                'seed': seed,
                'tvc': res_simple['score'][0],
                'cpc_percent': res_simple['score'][1],
                'cpc_avg': res_simple['score'][2]
            })
            # start = time.time()

            # Robust bid
            res_robust = autobidder_check(
                bidder=RobustBidMSE_CTRCVR,
                params={"input_campaigns": campaigns_path,
                        "input_stats": STATS_PATH_NOISED,
                        'eps_ctr': eps_ctr,
                        'eps_cvr': eps_cvr,
                        'gamma': 1.,
                        'u_0': 1.,
                        'LP': True,
                        'CPC': CPC,
                        'T': T
                        },
                loss_type=loss_type
            )
            # end = time.time()
            # robust_times.append(end-start)
            times_campaigns_robust = sum(res_robust['mean_times']) / len(res_robust['mean_times'])
            robust_times.append(times_campaigns_robust)

            metrics_list.append({
                'eps_ctr': eps_ctr,
                'eps_cvr': eps_cvr,
                'bidder_type': 'robust',
                'seed': seed,
                'tvc': res_robust['score'][0],
                'cpc_percent': res_robust['score'][1],
                'cpc_avg': res_robust['score'][2]
            })
            # T_times[T] = sum(robust_times) / len(robust_times)
            # T_times_non_robust[T] = sum(non_robust_times) / len(non_robust_times)
metrics_df = pd.DataFrame(metrics_list, columns=['eps_ctr', 'eps_cvr', 'bidder_type', 'seed', 'tvc', 'cpc_percent', 'cpc_avg'])

  0%|          | 0/5 [00:00<?, ?it/s]/Users/easolodneva/Documents/GitHub/robust-autobidding-noisy-conversions/.venv/lib/python3.11/site-packages/cvxpy/problems/problem.py:1481: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
  0%|          | 0/5 [00:00<?, ?it/s]/Users/easolodneva/Documents/GitHub/robust-autobidding-noisy-conversions/.venv/lib/python3.11/site-packages/cvxpy/problems/problem.py:1481: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
  0%|          | 0/5 [00:00<?, ?it/s]/Users/easolodneva/Documents/GitHub/robust-autobidding-noisy-conversions/.venv/lib/python3.11/site-packages/cvxpy/problems/problem.py:1481: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.w

In [24]:
T_r_ms = {k: v * 1e6 for k, v in T_times.items()}
T_r_ms

{100000: 8399.893104965844}

In [21]:
T_nr_ms = {k: v * 1e6 for k, v in T_times_non_robust.items()}
T_nr_ms

{1: 1706.9458356734385,
 10: 1691.3340606539293,
 100: 1728.2422990362843,
 1000: 1707.752529945456,
 10000: 1731.9220244154283,
 100000: 1711.8722104735716}

In [ ]:
sum(robust_times) / len(robust_times) / 317

In [12]:
agg_metrics = metrics_df.groupby(['eps_ctr', 'eps_cvr', 'bidder_type']).agg(
    mean_tvc=('tvc', 'mean'),
    std_tvc=('tvc', 'std'),
    mean_cpc_percent=('cpc_percent', 'mean'),
    std_cpc_percent=('cpc_percent', 'std'),
    mean_cpc_avg=('cpc_avg', 'mean'),
    std_cpc_avg=('cpc_avg', 'std')
).reset_index()

agg_metrics

,eps_ctr,eps_cvr,bidder_type,mean_tvc,std_tvc,mean_cpc_percent,std_cpc_percent,mean_cpc_avg,std_cpc_avg
0,0.00001,0.00001,robust,2.533994,0.000263,0.0,0.0,89.059590,0.007814
1,0.00001,0.00001,simple,1.709728,0.003388,0.0,0.0,353.646266,0.285041
2,0.00001,0.00010,robust,2.537290,0.000257,0.0,0.0,89.549034,0.018512
3,0.00001,0.00010,simple,1.709728,0.003388,0.0,0.0,353.646266,0.285041
4,0.00001,0.00100,robust,2.538282,0.000189,0.0,0.0,89.243812,0.008504
5,0.00001,0.00100,simple,1.709728,0.003194,0.0,0.0,353.646266,0.268739
6,0.00001,0.01000,robust,2.528044,0.000962,0.0,0.0,84.370863,0.027729
7,0.00001,0.01000,simple,1.709728,0.003388,0.0,0.0,353.646266,0.285041
8,0.00001,0.10000,robust,2.534149,0.004381,0.0,0.0,89.080637,0.089994
9,0.00001,0.10000,simple,1.709728,0.003388,0.0,0.0,353.646266,0.285041


In [13]:
METRICS_SAVE_PATH = f'../results/metrics_{loss_type.lower()}_BAT_CTR_CVR.csv'

In [14]:
agg_metrics.to_csv(METRICS_SAVE_PATH)

In [ ]:
plot_metric_with_error(
    agg_metrics=agg_metrics,
    metric_mean_col='mean_tvc',
    metric_std_col='std_tvc',
    metric_name='TVC',
    y_label='Total Value Clicks',
    loss_type=loss_type
)

# plot_metric_with_error(
#     agg_metrics=agg_metrics,
#     metric_mean_col='mean_cpc_percent',
#     metric_std_col='std_cpc_percent',
#     metric_name='CPC Percent',
#     y_label='Cost per Click (%)',
#     loss_type=loss_type
# )

# plot_metric_with_error(
#     agg_metrics=agg_metrics,
#     metric_mean_col='mean_cpc_avg',
#     metric_std_col='std_cpc_avg',
#     metric_name='CPC Avg',
#     y_label='Average Cost per Click',
#     loss_type=loss_type
# )

In [ ]:
plot_2d_heatmaps_interpolated(METRICS_SAVE_PATH, '1')